In [1]:
import pandas as pd
from tqdm import tqdm
from FlagEmbedding import FlagModel
from opensearchpy import OpenSearch

df_evaluation = pd.read_csv('evaluation.csv')
data_list = df_evaluation.values.tolist()

mrr_sum = 0

index_name = 'abstracts_bge'

model = FlagModel('BAAI/bge-large-en-v1.5', 
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=False)

client = OpenSearch(
    hosts=[{"host": 'opensearch', "port": 9200}],
    http_auth=('admin', 'admin'),
    use_ssl=True,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
)

for query, _, correct_pmid in tqdm(data_list):
    embedding = model.encode_queries(query).tolist()
    body = {
        "size": 5,
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "knn_score",
                    "lang": "knn",
                    "params": {
                        "field": "vector",
                        "query_value": embedding,
                        "space_type": "cosinesimil"
                    }   
                }
            }
        }
    }
    res = client.search(index=index_name, body=body)
    search_results_pmids = [hit['_source']['pmid'] for hit in res['hits']['hits']]
    if correct_pmid in search_results_pmids:
        rank = search_results_pmids.index(correct_pmid) + 1
        mrr_sum += 1 / rank

mrr = mrr_sum / len(data_list)
print(f"Mean Reciprocal Rank (MRR): {mrr}")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:28<00:00,  6.73it/s]

Mean Reciprocal Rank (MRR): 0.7259333333333335


In [6]:
import pandas as pd
from tqdm import tqdm
from FlagEmbedding import FlagModel
from opensearchpy import OpenSearch

df_evaluation = pd.read_csv('evaluation.csv')
data_list = df_evaluation.values.tolist()

mrr_sum = 0

index_name = 'abstracts_bge_fin1'

model = FlagModel('bge_large_fin', 
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=False)

client = OpenSearch(
    hosts=[{"host": 'opensearch', "port": 9200}],
    http_auth=('admin', 'admin'),
    use_ssl=True,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
)

for query, _, correct_pmid in tqdm(data_list):
    embedding = model.encode_queries(query).tolist()
    body = {
        "size": 5,
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "knn_score",
                    "lang": "knn",
                    "params": {
                        "field": "vector",
                        "query_value": embedding,
                        "space_type": "cosinesimil"
                    }   
                }
            }
        }
    }
    res = client.search(index=index_name, body=body)
    search_results_pmids = [hit['_source']['pmid'] for hit in res['hits']['hits']]
    if correct_pmid in search_results_pmids:
        rank = search_results_pmids.index(correct_pmid) + 1
        mrr_sum += 1 / rank

mrr = mrr_sum / len(data_list)
print(f"Mean Reciprocal Rank (MRR): {mrr}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [08:15<00:00,  2.02it/s]

Mean Reciprocal Rank (MRR): 0.7960666666666668
